In [2]:
import math
import random
import time
import matplotlib.pyplot as plt 
import datetime
from multiprocessing import Pool
from itertools import combinations, product
from statistics import mean, stdev
from itertools import combinations
from numpy import log, append
from collections import deque # Cola FIFO para la lista tabú

verbose = False

# Funciones comunes

In [1]:
def leeFichero(nombreFichero ='tsp/a280.tsp'):
    rutaFichero = './tsp/'
    extension = '.tsp'
    fichero = open(rutaFichero + nombreFichero + extension, 'r')
    array_puntos = []

    # Saltamos las cabeceras del fichero #
    for i in range(6):
        fichero.readline()

    for line in fichero:
        try:
            array_temp = line.replace('   ', ' ').replace('  ', ' ').replace('\n', '').split(' ')
            array_temp = [float(item) for item in array_temp if item]

            if(len(array_temp) > 1):
                array_puntos.append(array_temp)
        except:
            pass

    fichero.close()

    return array_puntos

In [2]:
def getDistancia(p1, p2):
    xd = p1[1] - p2[1]
    yd = p1[2] - p2[2]
    
    dij = round(math.sqrt(xd*xd + yd*yd))
    
    return dij

In [ ]:
def getMDistancias(nodos):
   
    distancias = [[0]*len(nodos) for i in range(len(nodos))]
    
    for i in range(len(nodos)):
        for j in range(len(nodos)):
            
            if(i==j):
                dij = 0 # Distancia a sí mismo infinita
            else:
                dij = getDistancia(nodos[i], nodos[j])
                
            distancias[i][j] = dij
            
    return distancias

In [ ]:
def pintaCamino(camino, nodos, titulo):
    
    x = []
    y = []
    
    for i in range(0, len(camino)-1):
        
        x.append(nodos[camino[i]-1][1])
        y.append(nodos[camino[i]-1][2])
        
    plt.figure(figsize=(12,12))    
    plt.title(titulo)
    plt.scatter(x, y, color='green') 
    
    for i in range(len(x)):
        if(i < len(x)-1):
            plt.arrow(x[i], y[i], x[i+1] - x[i], y[i+1] - y[i], head_width=1, length_includes_head=True)
        else:
             plt.arrow(x[i], y[i], x[0] - x[i], y[0] - y[i], head_width=1, length_includes_head=True)
                
    # Punto rojo inicial
    plt.plot(x[0],y[0], marker='o',
     markerfacecolor='red', markersize=12)
                

In [3]:
def getCosteCamino(camino, distancias, verbose=False):
    coste = 0
    
    for i in range(0, len(camino)-1):
        dij = distancias[camino[i]][camino[i+1]]
        coste += dij # Sumamos la distancia de un nodo al siguiente
        if(verbose):
            print(f"Acumulando Distancia {camino[i]} -> {camino[i+1]} = {dij}")
        
        
    dij = distancias[camino[-1]][camino[0]] #Sumamos la distancia del ultimo al primero
    coste += dij
    if(verbose):
            print(f"Acumulando Distancia {camino[-1]} -> {camino[0]} = {dij}")
    
    return coste

In [4]:
def getCostePivotes(distancias, caminoAnt, costeAnt, piv1, piv2):
      
    caminoAnt1 = [caminoAnt[piv1-1], caminoAnt[piv1]]
    if(piv1 != len(distancias)-1):
        caminoAnt1.append(caminoAnt[piv1+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt1.append(caminoAnt[0])
        
    caminoAnt2 = [caminoAnt[piv2-1], caminoAnt[piv2]]
    if(piv2 != len(distancias)-1):
        caminoAnt2.append(caminoAnt[piv2+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt2.append(caminoAnt[0])
        
    caminoNuevo1 = caminoAnt1.copy()
    if caminoAnt[piv2] in caminoNuevo1:
        caminoNuevo1[caminoNuevo1.index(caminoAnt[piv2])] = caminoAnt[piv1]
    caminoNuevo1[1] = caminoAnt[piv2]
    
    
    caminoNuevo2 = caminoAnt2.copy()
    if caminoAnt[piv1] in caminoNuevo2:
        caminoNuevo2[caminoNuevo2.index(caminoAnt[piv1])] = caminoAnt[piv2]
    caminoNuevo2[1] = caminoAnt[piv1]
   
    #print("SubCaminos anteriores: ", caminoAnt1, caminoAnt2)
    #print("SubCaminos nuevos: ", caminoNuevo1, caminoNuevo2)
    
    return (costeAnt - distancias[caminoAnt1[0]][caminoAnt1[1]] - distancias[caminoAnt1[1]][caminoAnt1[2]] - distancias[caminoAnt2[0]][caminoAnt2[1]] - distancias[caminoAnt2[1]][caminoAnt2[2]] + distancias[caminoNuevo1[0]][caminoNuevo1[1]] +  distancias[caminoNuevo1[1]][caminoNuevo1[2]] + distancias[caminoNuevo2[0]][caminoNuevo2[1]] + distancias[caminoNuevo2[1]][caminoNuevo2[2]])
    

# Algoritmo Greedy

In [8]:
# Algoritmo Greedy
def algoritmoGreedy(distancias):
    
    caminoSolucion = [0] # Empezamos en el primer indice de nodo
    nodosPendientes = list(range(1,len(distancias[1]))) # Lista del segundo al ultimo indice de nodo
    
    while len(nodosPendientes) > 0: # Mientras queden nodos por visitar

        nodoActual = caminoSolucion[-1] # Partimos desde el útlimo nodo añadido
        minDistancia = distancias[nodoActual][nodosPendientes[0]] # Inicializamos la distancia del actual al primer pendiente
        nodoMinDistancia = nodosPendientes[0]
        
        for nodoVecino in nodosPendientes: # Recorremos todos los nodos pendientes y nos quedamos con el de menor distancia
            d = distancias[nodoActual][nodoVecino]
            
            if d < minDistancia:
                minDistancia = d
                nodoMinDistancia = nodoVecino
                
        caminoSolucion.append(nodoMinDistancia)
        nodosPendientes.remove(nodoMinDistancia)
        
    return caminoSolucion
    #return [indice+1 for indice in caminoSolucion] # + 1 a cada elemento (tiene que empezar por 1)

## Búsqueda Aleatoria

In [5]:
# Busqueda aleatoria: el mejor camino de entre 1600n soluciones
def busquedaAleatoria(distancias, semilla, nIteraciones):
    
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    mejorCamino = []
    mejorCoste = -1
    
    for i in range (nIteraciones):
        caminoSolucion = [random.randint(1,len(distancias))] # Empezamos en un nodo aleatorio
        nodosPendientes = [1+indice for indice in range(len(distancias))] # Lista del 1 al num. nodos
        nodosPendientes.remove(caminoSolucion[0]) # Eliminamos el primer nodo visitado

        while len(nodosPendientes) > 0: # Mientras queden nodos por visitar

            nodoActual = caminoSolucion[-1] # Partimos desde el útlimo nodo añadido
            indiceAleatorio = random.randint(0,len(nodosPendientes)-1)
            nodoSiguiente = nodosPendientes[indiceAleatorio]

            caminoSolucion.append(nodoSiguiente)
            nodosPendientes.pop(indiceAleatorio)
        
        # Calculamos el coste de esa solucion
        costeActual = getCosteCamino(caminoSolucion, distancias)
        
        # La primera vez o si se reduce el coste, nos quedamos con la solucion
        if(mejorCoste==-1 or costeActual < mejorCoste): 
            mejorCamino = caminoSolucion
            mejorCoste = costeActual
        
    return [mejorCamino, mejorCoste, semilla]

In [6]:
def busquedaAleatoria_v2(distancias, semilla, nIteraciones):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    numNodos = len(distancias)
    
    # Solución inicial
    mejorCamino = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    mejorCoste = getCosteCamino(mejorCamino, distancias)
    
    for i in range (nIteraciones):
        
        caminoCandidato = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
        # Calculamos el coste de esa solucion
        costeActual = getCosteCamino(caminoCandidato, distancias)
        
        # Si se reduce el coste, nos quedamos con la solucion
        if(costeActual < mejorCoste): 
            mejorCamino = caminoCandidato
            mejorCoste = costeActual
        
    return [mejorCamino, mejorCoste, semilla]

## Búsqueda Local del Mejor Vecino

In [7]:
# Obtiene una lista de todos los caminos vecinos que se obtienen al permutar dos elementos del camino dado
def solucionesVecinas(camino):
    numNodos = len(camino)
    indices = list(range(numNodos))
    vecinos = []


    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = combinations(indices, 2)  

    # Aplicamos operador opt-2
    for cambio in comb:
        camino_new = camino.copy()

        # Permutamos los dos elementos
        camino_new[cambio[0]], camino_new[cambio[1]] = camino_new[cambio[1]], camino_new[cambio[0]]

        vecinos.append(camino_new)
    
    return vecinos

In [8]:
def busquedaLocalMejorVecino(distancias, semilla, nIteraciones, verbose=False):
     
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
  
    numNodos = len(distancias)
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    
    i=0
    
    repetir = True
    while repetir:
        
        mejorVecino = solucionActual.copy()
        costeMejorVecino = costeActual
        
        for solucionVecina in solucionesVecinas(solucionActual):
            
            costeSolucionVecina = getCosteCamino(solucionVecina, distancias)
            i+=1
            
            if(costeSolucionVecina < costeMejorVecino):
                mejorVecino = solucionVecina
                costeMejorVecino = costeSolucionVecina
                    
                
        if(costeMejorVecino < costeActual):
            solucionActual = mejorVecino.copy()
            costeActual = costeMejorVecino
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
            print("No mejora la exploración de vecinos con i: %i !" % i)
            repetir = False
            
        
        if(i > nIteraciones):
            print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False
        
            
    return [solucionActual, costeActual, i, semilla]

In [ ]:
def busquedaLocalMejorVecino_v2(distancias, semilla, nIteraciones):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    
    numNodos = len(distancias)
    indices = list(range(numNodos)) # Conversión a list para poderlo iterar +1 vez
    
    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = list(combinations(indices, 2))  
    
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    i=0
    swaps = 0
    repetir = True
    while repetir:
        mejorVecino = solucionActual
        costeMejorVecino = costeActual
        

    
        # Por cada posible camino vecino
        for cambio in comb:
            
            costeSolucionVecina = getCostePivotes(distancias, solucionActual, costeActual, cambio[0], cambio[1])
            #print("cambio " + str(cambio[0]) + " ; " + str(cambio[1]))
            i+=1

            if(costeSolucionVecina < costeMejorVecino):
                #print("Mejora la solucion vecina %f < %f" % (costeSolucionVecina, costeMejorVecino) )
                
                # Aplicamos operador opt-2
                camino_new = solucionActual.copy()
                camino_new[cambio[0]], camino_new[cambio[1]] = camino_new[cambio[1]], camino_new[cambio[0]]
                
                mejorVecino = camino_new
                costeMejorVecino = costeSolucionVecina
                
                
        if(costeMejorVecino < costeActual):
            #print("Mejora la solucion actual %f < %f" % (costeMejorVecino, costeActual) )
            solucionActual = mejorVecino
            costeActual = costeMejorVecino
            swaps+=1
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
            print("No mejora coste con i: %i !" % i)
            repetir = False
            
        if(i > nIteraciones):
            print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False   
    print("swaps: ", swaps)
    return [solucionActual, costeActual, i, semilla]

In [9]:
def busquedaLocalPrimerMejorVecino(distancias, semilla, nIteraciones, verbose=False):
     
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    
    numNodos = len(distancias)
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    i=0
    
    repetir = True
    while repetir:
        
        mejorVecino = solucionActual
        costeMejorVecino = costeActual
        
        solVecinas = solucionesVecinas(solucionActual) 

        
        for solucionVecina in solVecinas:
            
            #t = time.time()
            costeSolucionVecina = getCosteCamino(solucionVecina, distancias)  # hacer funcion coste reducida cambiando dos posiciones
            #elapsed = time.time() - t
            #print("Tiempo calcular coste solucion vecina: %f\n" % elapsed )
            i+=1
            
            if(costeSolucionVecina < costeMejorVecino):
                print("Mejora la solucion vecina %f < %f" % (costeSolucionVecina, costeMejorVecino) )
                mejorVecino = solucionVecina
                costeMejorVecino = costeSolucionVecina
                break # Paramos en el primer mejor vecino 
                
                
        if(costeMejorVecino < costeActual):
            print("Mejora la solucion actual %f < %f" % (costeMejorVecino, costeActual) )
            solucionActual = mejorVecino
            costeActual = costeMejorVecino
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
            print("No mejora coste con i: %i !" % i)
            repetir = False
            
        if(i > nIteraciones):
            print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False   
        
    return [solucionActual, costeActual, i, semilla]

In [10]:
def busquedaLocalPrimerMejorVecino_v2(distancias, semilla, nIteraciones):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    
    numNodos = len(distancias)
    indices = list(range(numNodos))
    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = list(combinations(indices, 2)) # Conversión a list para poderlo iterar +1 vez
    
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    i=0
    swaps = 0
    repetir = True
    while repetir:
        mejorVecino = solucionActual
        costeMejorVecino = costeActual
        


        # Por cada posible camino vecino
        for cambio in comb:
            
            costeSolucionVecina = getCostePivotes(distancias, solucionActual, costeActual, cambio[0], cambio[1])
            i+=1

            if(costeSolucionVecina < costeMejorVecino):
                #print("Mejora la solucion vecina %f < %f" % (costeSolucionVecina, costeMejorVecino) )
                
                # Aplicamos operador opt-2
                camino_new = mejorVecino.copy()
                camino_new[cambio[0]], camino_new[cambio[1]] = camino_new[cambio[1]], camino_new[cambio[0]]
                
                mejorVecino = camino_new
                costeMejorVecino = costeSolucionVecina
                break # Paramos en el primer mejor vecino 
                
                
        if(costeMejorVecino < costeActual):
            #print("Mejora la solucion actual %f < %f" % (costeMejorVecino, costeActual) )
            solucionActual = mejorVecino
            costeActual = costeMejorVecino
            swaps+=1
            
        # Condición de parada
        elif(costeMejorVecino >= costeActual):
            #print("No mejora coste con i: %i !" % i)
            repetir = False
            
        if(i > nIteraciones):
            print("Alcanzado limite de iteraciones: %i !" % i)
            repetir = False   
    
    print("swaps: ", swaps)    
    return [solucionActual, costeActual, i, semilla]
    

# Enfriamiento Simulado

In [21]:
def algoritmoEnfriamientoSimulado(distancias, semilla, nIteraciones, verbose=False):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    numNodos = len(distancias)
    indices = list(range(numNodos))
    # Todas las combinaciones de dos elementos (cambios posibles)
    comb = list(combinations(indices, 2))
    
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    
    solucionElite = solucionActual.copy() # Solución Best a devolver
    costeElite = costeActual
    
    # Representación gráfica coste
    if verbose:
        fig,ax = plt.subplots(1,1)
        ax.set_xlabel('Iteraciones') ; ax.set_ylabel('Coste Solución Actual')
        ax.set_xlim(0,nIteraciones) ; 
        X, Y = [], []
        
        ax.plot(X,Y)
        fig.canvas.draw()

    
    # Temparatura inicial
    # coste de algoritmo greedy (buena solución de inicio)
    costeGreedy = getCosteCamino(algoritmoGreedy(distancias), distancias)
    Tini = 0.3 / (-1 * log(0.3)) * costeGreedy
    T = Tini
    Lt = 20 # num soluciones en cada temparatura
    
    i=0
    
    while i < nIteraciones: # Condición de parada
        

        cambiosVisitados = []
        
        # Mejora: numero de vecinos dinamico hasta que acepte x vecinos
        for j in range(Lt): # Velocidad de enfriamiento
            
            # Seleccionamos un cambio aleatorio para generar un vecino
            cambio = comb[random.randint(0, len(comb) - len(cambiosVisitados) - 1)]
            
            #print("Cambio: ", cambio)
        
            while cambio in cambiosVisitados:
                #print("Cambio Repetido")
                # Volvemos a generar otro si ya está repetido
                cambio = comb[random.randint(0, len(comb) - len(cambiosVisitados) - 1)]
                
            cambiosVisitados.append(cambio)

            # Aplicamos operador opt-2 y evaluamos el coste           
            costeCandidata = getCostePivotes(distancias, solucionActual, costeActual, cambio[0], cambio[1])
            
            difCoste = costeCandidata - costeActual # Diferencia de costos
            
            # Aplicación del criterio de aceptación
            if difCoste < 0 or random.uniform(0,1) < math.exp(-difCoste / T):
                if verbose:
                    print("Solucion aceptada, coste = ", costeCandidata)
                # Generamos la solución candidata aplicando opt-2
                solucionActual[cambio[0]], solucionActual[cambio[1]] = solucionActual[cambio[1]], solucionActual[cambio[0]]
                costeActual = costeCandidata
                
                if costeActual < costeElite:
                    costeElite = costeActual
                    solucionElite = solucionActual.copy()
                
        # Mecanismo de enfriamiento: esquema de Cauchy
        T = Tini / (1 + i)
            
        i+=1 # Incrementamos el contador de iteraciones
        
        if verbose:
            X.append(i)
            Y.append(costeActual)
            
            ax.plot(X,Y)
            fig.canvas.draw()

        if verbose:
            print("T = %f \t i=%i" % (T, i))
        
    return [solucionElite, costeElite, i, semilla]
    

# Búsqueda Tabú

In [15]:
def busquedaTabu(distancias, semilla, nIteraciones, verbose=False):
    random.seed = semilla # Aplicamos la semilla pasada por parámetro
    numNodos = len(distancias)
    indices = list(range(0,numNodos))
    # Todas las combinaciones de dos elementos (cambios posibles)
    movimientos = list(combinations(indices, 2))
    
    # Solución de la que parte la (re)inicialización
    solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
    costeActual = getCosteCamino(solucionActual, distancias) 
    # Solución a devolver
    solucionElite = solucionActual.copy() 
    costeElite = costeActual
    
    listaTabu = deque(maxlen = int(numNodos/2)) # Cola FIFO de capacidad inicial n/2
    memoriaFrecuencias = [[0]*numNodos for i in range(numNodos)] # Matriz acumula posiciones x nodos 
    

    for i in range(nIteraciones):
        
        costeMejorVecino = float('inf')
        mejorVecino = []
        movMejorVecino = []
        
        # Examinar 40 soluciones candidatas de entre los vecinos de la solución actual
        for mov in random.sample(movimientos,40):
            
            # Calculamos el coste de la solucion candidata partiendo de la actual
            costeCandidata = getCostePivotes(distancias, solucionActual, costeActual, mov[0], mov[1])

            # Si es tabú, pero pasa el criterio de aspiración y mejora la mejor solución vecina actual
            # O no es tabú y mejora la mejor solución vecina actual
            if( (mov in listaTabu and costeCandidata < costeElite) or 
                 (not mov in listaTabu and costeCandidata < costeMejorVecino) ):
    
                    # Actualizamos la mejor solución vecina aplicando el operador opt-2 sobre la solucion actual
                    mejorVecino = solucionActual.copy()
                    mejorVecino[mov[0]], mejorVecino[mov[1]] = mejorVecino[mov[1]], mejorVecino[mov[0]]
                    costeMejorVecino = costeCandidata
                    movMejorVecino = mov
                    
                    # Inrcrementamos la ocurrencia de cada nodo en cada posición en la matriz de frecuencias
                    for pos, nodo in enumerate(mejorVecino):
                        memoriaFrecuencias[pos][nodo] += 1
        
                    # Si el mejor vecino mejora la mejor solución Elite, la actualizamos
                    if costeMejorVecino < costeElite:
                        costeElite = costeMejorVecino
                        solucionElite = mejorVecino.copy()
                       

                    # Si el mejor vecino mejora la mejor solución actual, la actualizamos
                    if costeMejorVecino < costeActual:
                        costeActual = costeMejorVecino
                        solucionActual = mejorVecino.copy()
                        
        # Añadimos el movimiento del mejor vecino a la lista tabú
        # Automáticamente si está llena, elimina el elemento más antiguo
        listaTabu.append(movMejorVecino) 


        # REINICIALIZACIÓN
        # 4 reinicializaciones en total
        # Se cambia la solucionActual de la que parte la siguiente iteración del algoritmo
        if(i > 0 and i % (nIteraciones/5) == 0 ):
            
            # Limpiar lista tabú
            #listaTabu.
        
            # Estrategia de reinicialización según probabilidad
            r = random.uniform(0,1)

            # 25% reinicialización solución inicial aleatoria
            if(r < 0.25):
                solucionActual = random.sample(range(numNodos), numNodos) # Combinación aleatoria sin repeteción de nodos
                costeActual = getCosteCamino(solucionActual, distancias) 

                if verbose:
                    print("reinicialización solución inicial aleatoria en iteracion: " + str(i))
                    print(solucionActual)
                    print("Coste = ", str(costeActual))

            # 50% memoria a largo plazo solución greedy
            elif( r >= 0.25 and r < 0.75):

                solucionActual = []

                # Por cada posición del camino
                for pos in memoriaFrecuencias:

                    menorFrecuencia = float('inf')

                    # Por cada nodo posible en una posición
                    for nodo, frecuencia in enumerate(pos):
                        # Si se ha usado menos veces en esa posición y no ha sido ya añadido al camino
                        if( frecuencia < menorFrecuencia and not nodo in solucionActual):
                            # Actualizamos el nodo de menor uso para esa posición y su frecuencia
                            mejorNodo = nodo
                            menorFrecuencia = frecuencia

                    # Añadimos el nodo de menor frecuencia de uso en esa posición
                    solucionActual.append(mejorNodo) 

                costeActual = getCosteCamino(solucionActual, distancias)

                if verbose:
                    print("reinicialización memoria a largo plazo solución greedy en iteracion: " + str(i))
                    print(solucionActual)
                    print("Coste = ", str(costeActual))

            # 25% reinicialización desde mejor solución obtenida
            else:
                solucionActual = solucionElite.copy()
                costeActual = costeElite

                if verbose:
                    print("reinicialización desde mejor solución obtenida en iteracion: " + str(i))
                    print(solucionActual)
                    print("Coste = ", str(costeActual))
                    
            # 50% Probabilidad de aumentar/reducir tamaño de la lista tabú
            if(r < 0.5):
                
                nuevaListaTabu = deque(list(listaTabu), listaTabu.maxlen*2)
                listaTabu = nuevaListaTabu
                              
            else:
              
                nuevaListaTabu = deque(list(listaTabu), int(listaTabu.maxlen/2))
                listaTabu = nuevaListaTabu

    # F. numIteraciones
    return [solucionElite, costeElite, semilla]